In [76]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [77]:
# Installing PgAdmin4
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-27 23:45:55--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.31MB/s    in 0.7s    

2022-01-27 23:45:56 (1.31 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [78]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VideoGames").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [79]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
# Show DataFrame
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-01-31 00:08:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

In [80]:
# Count number of records in the dataset
print(f"Total number of reviews: {df.count()}")

Total number of reviews: 145431


# Transform Data to drop NaN values and duplicates and to macth SQL table

In [81]:
# Print how many nan values and duplicates where drop
before = df.count()
print(f"Total number of reviews before droping NaN values: {df.count()}")
df = df.dropna()

after_nan = df.count()
print(f"Total number of reviews after droping NaN values: {after_nan}")
print(f'Total number of reviews drop with NaN values: {before-after_nan}')

df = df.dropDuplicates()
after_d = df.count()
print(f"Total number of reviews before droping Duplicates values: {after_nan}")
print(f'Total number of reviews drop after Duplicates values: {after_d}')
print(f'Total number of reviews drop with Duplicates: {after_nan - after_d}')

Total number of reviews before droping NaN values: 145431
Total number of reviews after droping NaN values: 145425
Total number of reviews drop with NaN values: 6
Total number of reviews before droping Duplicates values: 145425
Total number of reviews drop after Duplicates values: 145425
Total number of reviews drop with Duplicates: 0


# Examine the Schema

In [82]:
# Print Schema to see column datatypes
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [83]:
# Creating review_id_df
review_id_table_df = df.select('review_id', "customer_id", "product_id", "product_parent","review_date")
print(f'Total number of reviews: {review_id_table_df.count()}')

review_id_table_df.show(5)

Total number of reviews: 145425
+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1EB82XGHB4AXH|   40847927|B003YCQF3C|     452532551|2015-01-31 00:08:00|
|R10DEIO121647A|   47628015|B00C7B0G9A|     872270797|2015-01-30 00:08:00|
|R35QUODBLL4EZY|   38321201|B00B7S3KYM|     186820002|2015-01-30 00:08:00|
|R1FWYLPPOALS93|   24626672|B00F4CEHNK|     341969535|2015-01-23 00:08:00|
| R8595SHEBU8G7|   15954061|B010KYDNDG|     835376637|2015-01-22 00:08:00|
+--------------+-----------+----------+--------------+-------------------+
only showing top 5 rows



In [84]:
# Creating products_df
products_df = df.select('product_id', "product_title")
print(f'Total number of products: {products_df.count()}')

products_df.show(5)

Total number of products: 145425
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003YCQF3C|Agatha Christie 4...|
|B00C7B0G9A|      BioWare Points|
|B00B7S3KYM|   Chaser [Download]|
|B00F4CEHNK| Xbox Live Gift Card|
|B010KYDNDG|Minecraft for PC/...|
+----------+--------------------+
only showing top 5 rows



In [85]:
# Creating customer_df
customer_df = df.select('customer_id')
customer_df = customer_df.groupBy('customer_id').count()
customer_df = customer_df.withColumnRenamed('count', 'customer_count')
print(f'Number of customerss: {customer_df.count()}')

customer_df.show(5)

Number of customerss: 113400
+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    7071542|             2|
|   42594074|             2|
|    8736130|             2|
|   37032409|             1|
|   51494648|             4|
+-----------+--------------+
only showing top 5 rows



In [86]:
# Creating vine_df
vine_df = df.select('review_id','star_rating','helpful_votes','total_votes','vine')
print(f'Number of vine: {vine_df.count()}')
vine_df.show(5)

Number of vine: 145425
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1EB82XGHB4AXH|          4|            0|          0|   N|
|R10DEIO121647A|          1|            1|          3|   N|
|R35QUODBLL4EZY|          4|            0|          0|   N|
|R1FWYLPPOALS93|          5|            0|          0|   N|
| R8595SHEBU8G7|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



# Load DataFrames to the Database in AWS

In [92]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-bigdatachallange-rodrigo.cyfk2avge4dd.us-west-1.rds.amazonaws.com:5432/assigment"
config = {"user":"postgres", 
          "password": "postgres2425", 
          "driver":"org.postgresql.Driver"}

# Write DataFrame to active_user table in RDS

In [93]:
review_id_table_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

Py4JJavaError: ignored

In [75]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
customer_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)